# Fixing Problems with the Golden Rule

In [2]:
import pandas as pd
from dfply import *

## The Golden Rule of Tabular Data

The **Golden Rule of Tabular Data** states that tables should be arranged with

* One variable per column
* One individual per row

Computer scientists call this [database normalization](https://en.wikipedia.org/wiki/Database_normalization); [Wickham (2014)](https://vita.had.co.nz/papers/tidy-data.pdf) calls this *tidy data*.

## Violations of the golden rule

Wickim identifies the following violations

* Column headers are values, not variable names.
* Multiple variables are stored in one column.
* Variables are stored in both rows and columns.
* Multiple types of observational units are stored in the same table.
* A single observational unit is stored in multiple tables.


## Example 1 - PEW Income Research
    
**Task:** Load the file `PEW_income_religion.csv`.  Identify the violation of the golden rule.


In [3]:
income = pd.read_csv('./data/PEW_income_religion.csv')
income.head(3)

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58


**Problem:** There are measurments in the the table heading

## Measurements in column labels?  Stack!

We can fix issues with informative column labels by stacking the data with `gather`

In [4]:
(income >>
  gather("income_range", 
         "count", 
         columns_from('<$10k')) >>
  head())

,religion,income_range,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don't know/refused,<$10k,15


## The `column_from` selection helper

* `dfply` selection helper.  
* Returns an `Intention`
* Expression for **all columns to the right** of the given column

Other helper functions described [here](https://github.com/kieferk/dfply#selection-filter-functions) and [Lecture 4.1.1](4_1_1_advanced_applications_of_dfply_select.ipynb).

In [5]:
columns_from('<$10k')

In [6]:
columns_from('<$10k').evaluate(income)

['<$10k', '$10-20k', '$20-30k', '$30-40k', '$40-50k', '$50-75k']

##  Example 2
    
Let's look at a dataset based on another example provided by Wickham.

In [7]:
weather = pd.read_csv('https://raw.githubusercontent.com/nickhould/tidy-data-python/master/data/weather-raw.csv')
weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN


## Data in the column labels

`d1`, `d2`, etc. are days

In [8]:
weather_stacked = (weather >> 
                   gather('day',
                          'measurement',
                         columns_from('d1')))
weather_stacked >> head

,id,year,month,element,day,measurement
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


## Still violates the rule!

Note that 
* There are a lot of meaningless columns with no actual mmeasurements
* the `element` $\rightarrow$ *variable names*.
* Should be separate columns

In [9]:
weather_stacked >> head

,id,year,month,element,day,measurement
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


## Solution: Unstack the column containing variable names

In [11]:
weather_fixed = (weather_stacked 
                 >> filter_by(~X.measurement.isna()) 
                 >> spread(X.element, X.measurement))
weather_fixed

,id,year,month,day,tmax,tmin
0,MX17004,2010,2,d2,NaN,14.4
1,MX17004,2010,2,d3,NaN,14.4
2,MX17004,2010,3,d5,32.1,14.2
3,MX17004,2010,2,d2,27.3,NaN
4,MX17004,2010,2,d3,24.1,NaN


## Problems that require we reshape the data

The following problems 
* Measurement information in column labels
* Measurments on the same individual across multiple rows

## <font color="red"> Exercise 2 </font>
    
**Task:** Load the `rochester_mins_max_temp_2018.csv` data, contains data weather data for Rochester, MN. that is available at the [DNR website](https://www.dnr.state.mn.us/climate/historical/lcd.html?loc=rst). Note that `SM` and `AV` stand for *sum* and *average*, respectively.

1. Identify that problem with the current format.
2. Use `gather` and `spread` to fix the issue.

In [190]:
min_max = pd.read_csv("./data/rochester_mins_max_temp_2018.csv")
min_max.head()

,Month,Stat,Max,Min
0,Jan,SM,709.0,196.0
1,Jan,AV,22.9,6.3
2,Feb,SM,675.0,184.0
3,Feb,AV,24.1,6.6
4,Mar,SM,1200.0,693.0


> *Your thoughts here

In [148]:
# Your code here.

## Up Next

Stuff